In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

In [2]:
# make sure pandas is version 1.0 or higher
# make sure networkx is verion 2.4 or higher
print(pd.__version__)
print(nx.__version__)

1.4.2
2.7.1


In [3]:
from ema_workbench import (Model, Policy, ema_logging, SequentialEvaluator, 
                           MultiprocessingEvaluator)
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation

def sum_over(*args):
    return sum(args)

In [4]:
ema_logging.log_to_stderr(ema_logging.INFO)

#choose problem formulation number, between 0-5
#each problem formulation has its own list of outcomes
dike_model, planning_steps = get_model_for_problem_formulation(3)

In [5]:
#enlisting uncertainties, their types (RealParameter/IntegerParameter/CategoricalParameter), lower boundary, and upper boundary
import copy
for unc in dike_model.uncertainties:
    print(repr(unc))
    
uncertainties = copy.deepcopy(dike_model.uncertainties)

CategoricalParameter('discount rate 0', [0, 1, 2, 3])
CategoricalParameter('discount rate 1', [0, 1, 2, 3])
CategoricalParameter('discount rate 2', [0, 1, 2, 3])
CategoricalParameter('A.1_Brate', [0, 1, 2])
CategoricalParameter('A.2_Brate', [0, 1, 2])
CategoricalParameter('A.3_Brate', [0, 1, 2])
CategoricalParameter('A.4_Brate', [0, 1, 2])
CategoricalParameter('A.5_Brate', [0, 1, 2])


In [6]:
#enlisting policy levers, their types (RealParameter/IntegerParameter), lower boundary, and upper boundary
for policy in dike_model.levers:
    print(repr(policy))
    
levers = copy.deepcopy(dike_model.levers)

In [7]:
#enlisting outcomes
for outcome in dike_model.outcomes:
    print(repr(outcome))

ScalarOutcome('A.1 Total Costs', variable_name=('A.1_Expected Annual Damage 0', 'A.1_Expected Annual Damage 1', 'A.1_Expected Annual Damage 2', 'A.1_Dike Investment Costs 0', 'A.1_Dike Investment Costs 1', 'A.1_Dike Investment Costs 2'), function=<function sum_over at 0x7fb9208451f0>)
ScalarOutcome('A.1_Expected Number of Deaths', variable_name=('A.1_Expected Number of Deaths 0', 'A.1_Expected Number of Deaths 1', 'A.1_Expected Number of Deaths 2'), function=<function sum_over at 0x7fb9208451f0>)
ScalarOutcome('A.2 Total Costs', variable_name=('A.2_Expected Annual Damage 0', 'A.2_Expected Annual Damage 1', 'A.2_Expected Annual Damage 2', 'A.2_Dike Investment Costs 0', 'A.2_Dike Investment Costs 1', 'A.2_Dike Investment Costs 2'), function=<function sum_over at 0x7fb9208451f0>)
ScalarOutcome('A.2_Expected Number of Deaths', variable_name=('A.2_Expected Number of Deaths 0', 'A.2_Expected Number of Deaths 1', 'A.2_Expected Number of Deaths 2'), function=<function sum_over at 0x7fb9208451f

In [8]:
#running the model through EMA workbench 
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=50, policies=4)

[MainProcess/INFO] pool started with 12 workers
[MainProcess/INFO] performing 50 scenarios * 4 policies * 1 model(s) = 200 experiments
100%|████████████████████████████████████████| 200/200 [01:10<00:00,  2.84it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [9]:
#observing the simulation runs
experiments, outcomes = results
print(outcomes.keys())
experiments

dict_keys(['A.1 Total Costs', 'A.1_Expected Number of Deaths', 'A.2 Total Costs', 'A.2_Expected Number of Deaths', 'A.3 Total Costs', 'A.3_Expected Number of Deaths', 'A.4 Total Costs', 'A.4_Expected Number of Deaths', 'A.5 Total Costs', 'A.5_Expected Number of Deaths', 'RfR Total Costs', 'Expected Evacuation Costs'])


,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,...,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,EWS_DaysToThreat,scenario,policy,model
0,15.0,113.479923,10,0.770136,339.310889,1.5,0.944292,317.342219,1.0,0.489705,...,7.0,10.0,2.0,5.0,7.0,9.0,4.0,4,0,dikesnet
1,12.0,122.145750,1.5,0.545091,296.998800,1.0,0.031960,140.299655,1.5,0.289509,...,7.0,10.0,2.0,5.0,7.0,9.0,4.0,5,0,dikesnet
2,22.0,109.633053,1.5,0.160546,138.337199,1.0,0.569033,78.663231,1.0,0.653688,...,7.0,10.0,2.0,5.0,7.0,9.0,4.0,6,0,dikesnet
3,95.0,140.117317,10,0.687769,112.053182,10,0.992838,171.124056,1.0,0.136491,...,7.0,10.0,2.0,5.0,7.0,9.0,4.0,7,0,dikesnet
4,121.0,68.913650,1.5,0.593934,51.065576,10,0.887263,154.243679,1.5,0.616494,...,7.0,10.0,2.0,5.0,7.0,9.0,4.0,8,0,dikesnet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,6.0,82.665778,1.5,0.330352,286.428045,10,0.519639,94.921411,1.5,0.817756,...,9.0,5.0,10.0,8.0,8.0,3.0,2.0,49,3,dikesnet
196,129.0,277.167399,1.0,0.040659,91.453355,10,0.449499,254.962483,10,0.211070,...,9.0,5.0,10.0,8.0,8.0,3.0,2.0,50,3,dikesnet
197,29.0,182.734680,10,0.856335,56.599540,1.5,0.273333,264.926017,1.5,0.584787,...,9.0,5.0,10.0,8.0,8.0,3.0,2.0,51,3,dikesnet
198,77.0,268.138544,1.5,0.704340,101.645862,1.5,0.978085,84.673096,10,0.637724,...,9.0,5.0,10.0,8.0,8.0,3.0,2.0,52,3,dikesnet


In [24]:
# only works because we have scalar outcomes
pd.DataFrame(outcomes)

,A.1 Total Costs,A.1_Expected Number of Deaths,A.2 Total Costs,A.2_Expected Number of Deaths,A.3 Total Costs,A.3_Expected Number of Deaths,A.4 Total Costs,A.4_Expected Number of Deaths,A.5 Total Costs,A.5_Expected Number of Deaths,RfR Total Costs,Expected Evacuation Costs
0,7.122315e+07,0.0,1.740494e+08,0.000000,1.526233e+08,0.0,5.303178e+07,0.0,1.684455e+08,0.0,1.195400e+09,0.000000
1,7.122315e+07,0.0,2.920048e+08,0.011546,1.526233e+08,0.0,5.303178e+07,0.0,1.684455e+08,0.0,1.195400e+09,7708.224766
2,7.122315e+07,0.0,1.745589e+08,0.000057,1.526233e+08,0.0,5.303178e+07,0.0,1.684455e+08,0.0,1.195400e+09,31.302932
3,7.122315e+07,0.0,1.740494e+08,0.000000,1.526233e+08,0.0,5.303178e+07,0.0,1.684455e+08,0.0,1.195400e+09,0.000000
4,7.122315e+07,0.0,1.740494e+08,0.000000,1.526233e+08,0.0,5.303178e+07,0.0,1.684455e+08,0.0,1.195400e+09,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
195,2.621674e+08,0.0,2.757740e+08,0.000000,1.006469e+08,0.0,6.574913e+07,0.0,1.638319e+08,0.0,1.057000e+09,0.000000
196,2.621674e+08,0.0,2.757740e+08,0.000000,1.006469e+08,0.0,6.574913e+07,0.0,1.638319e+08,0.0,1.057000e+09,0.000000
197,2.621674e+08,0.0,2.757740e+08,0.000000,1.006469e+08,0.0,6.574913e+07,0.0,1.638319e+08,0.0,1.057000e+09,0.000000
198,2.621674e+08,0.0,2.757740e+08,0.000000,1.006469e+08,0.0,6.574913e+07,0.0,1.638319e+08,0.0,1.057000e+09,0.000000


In [20]:
# defining specific policies
# for example, policy 1 is about extra protection in upper boundary
# policy 2 is about extra protection in lower boundary
# policy 3 is extra protection in random locations

def get_do_nothing_dict():
    return {l.name:0 for l in dike_model.levers}


policies = [Policy('policy 1', **dict(get_do_nothing_dict(), 
                                    **{'0_RfR 0':1,
                                      '0_RfR 1':1,
                                      '0_RfR 2':1,
                                      'A.1_DikeIncrease 0':5})),
           Policy('policy 2', **dict(get_do_nothing_dict(), 
                                   **{'4_RfR 0':1,
                                      '4_RfR 1':1,
                                      '4_RfR 2':1,
                                      'A.5_DikeIncrease 0':5})),
           Policy('policy 3', **dict(get_do_nothing_dict(),
                                   **{'1_RfR 0':1,
                                      '2_RfR 1':1,
                                      '3_RfR 2':1,
                                      'A.3_DikeIncrease 0':5}))]

In [21]:
#pass the policies list to EMA workbench experiment runs
n_scenarios = 100
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(n_scenarios,
                                            policies)

[MainProcess/INFO] pool started with 12 workers
[MainProcess/INFO] performing 100 scenarios * 3 policies * 1 model(s) = 300 experiments
100%|████████████████████████████████████████| 300/300 [01:48<00:00,  2.78it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


,A.1 Total Costs,A.1_Expected Number of Deaths,A.2 Total Costs,A.2_Expected Number of Deaths,A.3 Total Costs,A.3_Expected Number of Deaths,A.4 Total Costs,A.4_Expected Number of Deaths,A.5 Total Costs,A.5_Expected Number of Deaths,RfR Total Costs,Expected Evacuation Costs
0,7.122315e+07,0.0,1.740494e+08,0.000000,1.526233e+08,0.0,5.303178e+07,0.0,1.684455e+08,0.0,1.195400e+09,0.000000
1,7.122315e+07,0.0,2.920048e+08,0.011546,1.526233e+08,0.0,5.303178e+07,0.0,1.684455e+08,0.0,1.195400e+09,7708.224766
2,7.122315e+07,0.0,1.745589e+08,0.000057,1.526233e+08,0.0,5.303178e+07,0.0,1.684455e+08,0.0,1.195400e+09,31.302932
3,7.122315e+07,0.0,1.740494e+08,0.000000,1.526233e+08,0.0,5.303178e+07,0.0,1.684455e+08,0.0,1.195400e+09,0.000000
4,7.122315e+07,0.0,1.740494e+08,0.000000,1.526233e+08,0.0,5.303178e+07,0.0,1.684455e+08,0.0,1.195400e+09,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
195,2.621674e+08,0.0,2.757740e+08,0.000000,1.006469e+08,0.0,6.574913e+07,0.0,1.638319e+08,0.0,1.057000e+09,0.000000
196,2.621674e+08,0.0,2.757740e+08,0.000000,1.006469e+08,0.0,6.574913e+07,0.0,1.638319e+08,0.0,1.057000e+09,0.000000
197,2.621674e+08,0.0,2.757740e+08,0.000000,1.006469e+08,0.0,6.574913e+07,0.0,1.638319e+08,0.0,1.057000e+09,0.000000
198,2.621674e+08,0.0,2.757740e+08,0.000000,1.006469e+08,0.0,6.574913e+07,0.0,1.638319e+08,0.0,1.057000e+09,0.000000


In [25]:
# only works because we have scalar outcomes
pd.DataFrame(results[1])

,A.1 Total Costs,A.1_Expected Number of Deaths,A.2 Total Costs,A.2_Expected Number of Deaths,A.3 Total Costs,A.3_Expected Number of Deaths,A.4 Total Costs,A.4_Expected Number of Deaths,A.5 Total Costs,A.5_Expected Number of Deaths,RfR Total Costs,Expected Evacuation Costs
0,5.397251e+07,0.000000,1.095201e+08,0.088571,1.344469e+09,1.977799,0.000000e+00,0.000000,0.000000e+00,0.000000,253800000.0,0.0
1,5.397251e+07,0.000000,1.734936e+08,0.189858,0.000000e+00,0.000000,3.073570e+07,0.016619,0.000000e+00,0.000000,253800000.0,0.0
2,5.397251e+07,0.000000,2.165553e+07,0.021914,1.725967e+08,0.319993,1.350450e+08,0.056430,0.000000e+00,0.000000,253800000.0,0.0
3,5.397251e+07,0.000000,1.876042e+08,0.234379,5.537974e+08,1.315890,0.000000e+00,0.000000,0.000000e+00,0.000000,253800000.0,0.0
4,5.397251e+07,0.000000,1.645147e+07,0.022348,3.129531e+07,0.079533,9.054602e+06,0.006697,4.350162e+08,0.532353,253800000.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
295,2.273052e+07,0.013232,6.056404e+08,0.455885,2.879840e+07,0.000000,1.659682e+07,0.005970,0.000000e+00,0.000000,369700000.0,0.0
296,1.972443e+08,0.175343,5.890743e+06,0.007580,2.879840e+07,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,369700000.0,0.0
297,0.000000e+00,0.000000,7.807540e+08,0.693078,2.879840e+07,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,369700000.0,0.0
298,1.270634e+08,0.066561,8.144099e+08,0.558885,2.879840e+07,0.000000,0.000000e+00,0.000000,1.981612e+06,0.001379,369700000.0,0.0
